### Get models

In [1]:
import credibility_model as cred

import pandas as pd
import requests
import json

from pyspark.sql.functions import col
from pyspark.sql import Row
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql import SparkSession

Fake_tokenizer is imported
Model_in_json2 is imported
LSTM model imported
Bigram is imported
Trigram is Imported
Pipeline Imported
xgb_obj_model Downloaded
dict_transformer_ht Downloaded
dict_transformer_hp Downloaded
dict_transformer_bt Downloaded
dict_transformer_bp Downloaded
count_vectorizer_h Downloaded
count_vectorizer_b Downloaded
Data Already Imported
Data Already Imported
DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
               sparse=True)


In [2]:
def get_data_from_db():
    host = 'https://search-solytics-tzxvnvrvkscgklaedz5lr3iqxu.ap-south-1.es.amazonaws.com/news_test_list/_search'
    json_body = '''{
        "query": {
                "bool": {
                    "must_not": {
                        "exists":{
                            "field":"sentiment_ML"
                            }
                        }
                    }
                }
    }'''
    headers = {
        'Content-Type': 'application/json',
    }
    params = {
        'size':30
    }
    resp = requests.get(host,params=params, headers=headers, data=json_body)
    resp_text = json.loads(resp.text)
    document_list = []

    # print(resp)
    for data in resp_text['hits']['hits']:
        content_list = {}
        content_list["id"] = data["_id"]
        content_list["content"] = data["_source"]["Content"]
        content_list["url"] = data["_source"]["URL"]
        content_list["source"] = data["_source"]["Source"]
        content_list["title"] = data["_source"]["Title"]
        document_list.append(content_list)
    return document_list

In [3]:
def spark_data():
    data = get_data_from_db()
    df = pd.DataFrame(data)
    spark_df = spark.createDataFrame(df)
    return spark_df

In [4]:
def credibility_score(df):
    
    def score(row):
        final_coeff = {
                                'obj_score_coeff' : 0.3,
                                'fake_score_coeff' : 0.35,
                                'bias_score_coeff' : 0.25,
                                'google_rank_coeff' : 0.1,
                                'alexa_rank_coeff': -1*0.000001
                            }

        # giving a default values to all the scores
        fake_score = 0.5
        obj_score = 0.5
        google_score = 0.5
        alexa_score = 0.5
        bias_score = 0.5

        fake_score = cred.model.fake_score(row.content)
        google_score, alexa_score = cred.model.google_alexa_score(row.url)
        bias_score = cred.model.bias_score(row.source)
        obj_score = cred.model.obj_score(row.title,row.content)

        final_score = fake_score[0,0]*final_coeff['fake_score_coeff'] \
                + bias_score*final_coeff['bias_score_coeff'] \
                    + google_score*final_coeff['google_rank_coeff'] \
                        +alexa_score * final_coeff['alexa_rank_coeff']\
                            + obj_score[0]*final_coeff['obj_score_coeff']  
        idd = row.id
        contentt = row.content
        sourcee = row.source
        titlee = row.title
        urll = row.url
        credibilityy = float(final_score)
        return Row(id=idd,content=contentt,source=sourcee,title=titlee,url=urll,credibility=credibilityy)
    
    data = df.rdd.map(score)
    schema = StructType([
        StructField("id",StringType(),False),
        StructField("content",StringType(),False),
        StructField("source",StringType(),False),
        StructField("title",StringType(),False),
        StructField("url",StringType(),False),
        StructField("credibility",FloatType(),False)
    ])
    result = spark.createDataFrame(data,schema)
    return result

In [6]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Word Count") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

NameError: name 'local' is not defined

In [6]:
data = get_data_from_db()
df = spark_data()

In [7]:
ans = credibility_score(df)

In [8]:
ans.show()

+---+--------------------+--------------------+--------------------+--------------------+-----------+
| id|             content|              source|               title|                 url|credibility|
+---+--------------------+--------------------+--------------------+--------------------+-----------+
|248|Page Content
JS-4...|    www.treasury.gov|Swiss Company, In...|https://www.treas...|  0.3200599|
|152|Why did this happ...|   www.bloomberg.com|    Are you a robot?|https://www.bloom...| 0.40053526|
|533|One of the spouse...|    nehandaradio.com|Aguy Georgias Arc...|https://nehandara...| 0.55483353|
|263|As Turkish forces...|www.justsecurity.org|“With a Little He...|https://www.justs...|  0.3256237|
|472|reasury action ta...|    mx.usembassy.gov|Treasury Sanction...|https://mx.usemba...| 0.37049803|
|404|John Arnold Brede...|    en.wikipedia.org|     John Bredenkamp|https://en.wikipe...| 0.44265172|
| 37|[Federal Register...|     www.govinfo.gov|Federal Register,...|https://www.go